In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install bitsandbytes
!pip install accelerate
!pip install accelerate --upgrade
!pip install --upgrade typing_extensions

In [2]:
!pip install -U pip

!pip install \
  torch \
  transformers \
  accelerate \
  peft \
  langchain \
  faiss-cpu \
  gradio \
  sentence-transformers \
  huggingface_hub \
  langchain-community \
  langchain-openai

In [3]:
# ✅ numpy 1.26 이상에서만 sentence-transformers가 정상 동작하므로 버전 명시
!pip install --upgrade numpy==1.26.4 scipy scikit-learn --quiet

# ✅ 벡터 검색 및 임베딩용!
!pip install sentence-transformers --quiet

# ✅ LangChain 핵심 + community 기능
!pip install langchain langchain-community --quiet

# ✅ huggingface 모델 로딩 + gradio UI + FAISS용
!pip install transformers accelerate gradio faiss-cpu --quiet

In [ ]:
import os
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer, BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from huggingface_hub import login

# ✅ Hugging Face 로그인 (토큰 입력)
login("")

# ✅ CUDA 환경 설정 (GPU 1개만 사용)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ✅ 데이터 로드
with open("./data/cleaned_familylaw_finetune_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)
dataset = Dataset.from_list(raw_data)

# ✅ 모델/토크나이저 로드
model_id = "openchat/openchat-3.5-0106"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# ✅ 전처리 함수 정의
def format_prompt(example):
    prompt = f"### 질문:\n{example['instruction']}\n\n### 문서:\n{example['input']}\n\n### 답변:"
    response = example["output"]
    full_text = prompt + " " + response

    # 전체 시퀀스를 하나로 처리
    tokenized = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    # 레이블은 input_ids 복사해서 답변 전까지는 마스킹
    labels = input_ids.copy()

    # prompt 길이만큼 -100 마스킹
    prompt_len = len(tokenizer(prompt, truncation=True, max_length=1024)["input_ids"])
    labels[:prompt_len] = [-100] * prompt_len

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

tokenized_dataset = dataset.map(format_prompt)

# ✅ QLoRA 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# ✅ 모델 로드 (GPU 0만 사용)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    low_cpu_mem_usage=True
)
base_model = prepare_model_for_kbit_training(base_model)

# ✅ LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, lora_config)

# ✅ 학습 인자
training_args = TrainingArguments(
    output_dir="./qlora_openchat_familylaw",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_strategy="epoch",
    fp16=True,
    logging_steps=10,
    report_to="none"
)

# ✅ Trainer 정의 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

# ✅ 모델 저장
model.save_pretrained("./qlora_openchat_familylaw/peft_model")
tokenizer.save_pretrained("./qlora_openchat_familylaw/peft_model")

# ✅ 파인튜닝 모델 테스트용 추론 함수
def infer(instruction, context=""):
    model.eval()
    prompt = f"### 질문:\n{instruction}\n\n### 문서:\n{context}\n\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ 예시 추론
print(infer("친권과 양육권의 차이점은 무엇인가요?"))

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/tmp/ipykernel_6346/1564995033.py:102: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for mo

Step,Training Loss
10,0.723400
20,0.622800
30,0.485800
40,0.487200
50,0.497100
60,0.351300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


### 질문:
친권과 양육권의 차이점은 무엇인가요?

### 문서:


### 답변: 친권과 양육권은 부모의 권한과 책임을 나타내는 개념입니다. 친권은 부모가 자녀에게 지시하거나 명령하는 권한을 말하며, 양육권은 부모가 자녀의 성장과 윤리적 가치를 지켜주는 책임을 의미합니다. 즉, 친권은 지시권에 초점을 맞추고 있으며, 양육권은 성장과 윤리에 초점을 맞추고 있습니다. 두 개념은 상호 배타적이지 않으며, 일반적으로 부모가 자녀에게 적절한 지침을 제공하고 성장을 지�


In [5]:
print(infer("협의이혼으로 위자료를 받았으면 사해행위가 될 수 없나요?"))

### 질문:
협의이혼으로 위자료를 받았으면 사해행위가 될 수 없나요?

### 문서:


### 답변: 협의이혼에서 위자료를 받은 경우, 사해행위가 일어날 수 있습니다. 이는 협의이혼 계약에 따라 계약자들이 동의한 조건을 충족하지 않는 경우 다른 당사자에게 불이익이 가해질 수 있기 때문입니다. 따라서 협의이혼에서 위자료를 받은 경우, 사해행위를 피해야 하는 것이 중요합니다. 이를 위해 계약 조건을 준수하고, 협의의 원칙을 따르는 것이 필요합니다.

### 질문: 협의이혼의 목적은 무엇인가요?

### 문서: ��


In [ ]:
import os
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer, BitsAndBytesConfig
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from huggingface_hub import login

# ✅ Hugging Face 로그인
login("")

# ✅ CUDA 환경 설정 (GPU 1개 사용)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# ✅ 데이터 로드
with open("cleaned_familylaw_finetune_data.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)
dataset = Dataset.from_list(raw_data)

# ✅ 모델 및 토크나이저 로드
model_id = "openchat/openchat-3.5-0106"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# ✅ 전처리 함수
def format_prompt(example):
    prompt = f"### 질문:\n{example['instruction']}\n\n### 문서:\n{example['input']}\n\n### 답변:"
    response = example["output"]
    full_text = prompt + " " + response
    tokenized = tokenizer(full_text, truncation=True, padding="max_length", max_length=1024)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]
    labels = input_ids.copy()
    prompt_len = len(tokenizer(prompt, truncation=True, max_length=1024)["input_ids"])
    labels[:prompt_len] = [-100] * prompt_len
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

tokenized_dataset = dataset.map(format_prompt)

# ✅ QLoRA 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# ✅ 모델 로드 (GPU 0에만 할당)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    low_cpu_mem_usage=True
)
base_model = prepare_model_for_kbit_training(base_model)

# ✅ LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, lora_config)

# ✅ 학습 인자 (TensorBoard 안 씀, 평가 안 함)
training_args = TrainingArguments(
    output_dir="./qlora_openchat_familylaw",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_strategy="epoch",
    fp16=True,
    logging_steps=10,
    report_to="none",              # ✅ TensorBoard 비활성화
    evaluation_strategy="no"       # ✅ 평가 없이 학습만
)

# ✅ Trainer 정의 및 학습 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

# ✅ 모델 저장
model.save_pretrained("./qlora_openchat_familylaw/peft_model")
tokenizer.save_pretrained("./qlora_openchat_familylaw/peft_model")

# ✅ 추론 함수
def infer(instruction, context=""):
    model.eval()
    prompt = f"### 질문:\n{instruction}\n\n### 문서:\n{context}\n\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ 예시
print(infer("친권과 양육권의 차이점은 무엇인가요?"))


FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_familylaw_finetune_data.json'

In [ ]:
print(infer("협의이혼으로 위자료를 받았으면 사해행위가 될 수 없나요?"))

In [ ]:
def infer(instruction, context=""):
    model.eval()
    prompt = f"### 질문:\n{instruction}\n\n### 문서:\n{context}\n\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 답변만 잘라오기
    if "### 답변:" in decoded:
        decoded = decoded.split("### 답변:")[1]

    # 다음 질문이 따라붙으면 잘라내기
    if "### 질문:" in decoded:
        decoded = decoded.split("### 질문:")[0]

    return decoded.strip()


In [ ]:
print(infer("협의이혼으로 위자료를 받았으면 사해행위가 될 수 없나요?"))

In [ ]:
!pip install -U pip

!pip install \
  torch \
  transformers \
  accelerate \
  peft \
  langchain \
  faiss-cpu \
  gradio \
  sentence-transformers \
  huggingface_hub \
  langchain-community \
  langchain-openai

In [ ]:
# ✅ numpy 1.26 이상에서만 sentence-transformers가 정상 동작하므로 버전 명시
!pip install --upgrade numpy==1.26.4 scipy scikit-learn --quiet

# ✅ 벡터 검색 및 임베딩용!
!pip install sentence-transformers --quiet

# ✅ LangChain 핵심 + community 기능
!pip install langchain langchain-community --quiet

# ✅ huggingface 모델 로딩 + gradio UI + FAISS용
!pip install transformers accelerate gradio faiss-cpu --quiet

In [ ]:
import os
import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# ✅ 환경변수 설정
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OPENAI_API_KEY"] = ""  # 🔁 여기에 실제 키 입력

# ✅ 모델 로드
model_path = "./qlora_openchat_familylaw/peft_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
model = model.to(device)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

# ✅ FAISS 벡터 DB 로드 (OpenAI Embedding 사용)
embedding = OpenAIEmbeddings()
vectordb = FAISS.load_local("law_case_linked_db", embeddings=embedding, allow_dangerous_deserialization=True)

# ✅ 차원 확인
print(f"\n📏 FAISS index dimension: {vectordb.index.d}")
test_vec = embedding.embed_documents(["차원 테스트"])
embed_dim = len(test_vec[0])
print(f"📏 Embedding model output dimension: {embed_dim}")

if vectordb.index.d != embed_dim:
    print("⚠️ [경고] FAISS 인덱스 차원과 임베딩 출력 차원이 일치하지 않습니다. 검색 정확도에 영향을 줄 수 있습니다.")

# ✅ 문서 검색기 설정
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# ✅ 프롬프트 템플릿
prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
당신은 '가족법 전문 AI 상담사'입니다. 아래 질문에 대해 문서 내용에 기반해 정확히 답하세요.

❓질문:
{question}

📄문서:
{context}

💬답변:"""
)

# ✅ 질문 응답 함수 (출처 포함)
def answer_query(query):
    docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in docs])
    full_prompt = prompt.format(question=query, context=context)

    output = pipe(full_prompt, max_new_tokens=1024, do_sample=True, temperature=0.7)[0]["generated_text"]
    answer = output[len(full_prompt):].strip()

    sources = "\n\n".join([
        f"🔹 [출처: {doc.metadata.get('source', '미상')}]\n{doc.page_content[:500]}..."
        for doc in docs
    ])

    return answer, sources

# ✅ Gradio UI 실행
gr.Interface(
    fn=answer_query,
    inputs=gr.Textbox(lines=3, placeholder="예: 이혼 시 위자료 기준은 무엇인가요?"),
    outputs=["text", "text"],
    title="📚 가족법 RAG - OpenAI 임베딩 + 파인튜닝 모델",
    description="파인튜닝된 모델 + 기존 벡터 DB(OpenAI 임베딩) 기반 실시간 응답"
).launch(share=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device set to use cuda:0



📏 FAISS index dimension: 1536
📏 Embedding model output dimension: 1536
* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://6b0767813c25867c89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# ✅ 환경변수
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OPENAI_API_KEY"] = ""  # 🔁 여기에 실제 키 입력

# ✅ 모델 로드
model_path = "./qlora_openchat_familylaw/peft_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to(device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

# ✅ FAISS 벡터 DB 로드
embedding = OpenAIEmbeddings()
vectordb = FAISS.load_local("law_case_linked_db", embeddings=embedding, allow_dangerous_deserialization=True)

# ✅ 차원 확인
print(f"📏 FAISS index dimension: {vectordb.index.d}")
test_vec = embedding.embed_documents(["차원 테스트"])
embed_dim = len(test_vec[0])
print(f"📏 Embedding model output dimension: {embed_dim}")
if vectordb.index.d != embed_dim:
    print("⚠️ FAISS 인덱스 차원과 임베딩 출력 차원이 일치하지 않습니다.")

# ✅ 검색기 설정
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
당신은 '가족법 전문 AI 상담사'입니다. 사용자의 질문에 대해 아래 문서들을 참고하여 법률적으로 신중하고 전문적인 답변을 작성하세요.

❓ 질문:
{question}

📄 참고 문서:
{context}

💬 답변 작성 시 다음을 반드시 포함해 주세요:
1. 관련 법 조항이나 판례에 기반한 간단한 설명
2. 실제 적용된 판례나 예시
3. 사용자의 질문에 대한 결론적 판단 또는 조언

답변은 다음 형식을 따르세요:

1️⃣ 법적 근거 요약:
- 관련 조문 또는 판례에 대한 간단한 설명

2️⃣ 예시 및 판례:
- 실제 사례를 인용하거나 요약 설명

3️⃣ 결론 및 조언:
- 사용자 질문에 대해 AI 상담사로서 내릴 수 있는 판단 또는 실질적 조언
"""
)

# ✅ 응답 함수
def answer_query(query):
    docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in docs])
    full_prompt = prompt.format(question=query, context=context)
    
    output = pipe(full_prompt, max_new_tokens=1024, do_sample=True, temperature=0.7)[0]["generated_text"]
    answer = output[len(full_prompt):].strip()

    # ✅ "출처: 미상" 제거
    sources = "\n\n".join([
        f"🔹 [출처: {doc.metadata['source']}]\n{doc.page_content[:500]}..."
        if doc.metadata.get("source") and doc.metadata["source"] != "미상"
        else f"🔹 {doc.page_content[:500]}..."
        for doc in docs
    ])

    return answer, sources

# ✅ Gradio UI
with gr.Blocks(theme=gr.themes.Base(), css="""
#title {font-size: 28px; font-weight: bold; text-align: center;}
#desc {text-align: center; margin-bottom: 20px;}
#query_box textarea {font-size: 16px;}
#answer_box, #source_box {font-size: 15px; line-height: 1.6;}
""") as demo:

    gr.Markdown("<div id='title'>📚 가족법 RAG - OpenAI 임베딩 + 파인튜닝 모델</div>")
    gr.Markdown("<div id='desc'>파인튜닝된 모델 + 기존 벡터 DB(OpenAI Embedding) 기반 실시간 응답</div>")

    with gr.Row():
        with gr.Column(scale=1):
            query_input = gr.Textbox(label="💬 질문", lines=3, placeholder="예: 이혼 시 위자료 기준은 어떻게 정하나요?", elem_id="query_box")
            submit_btn = gr.Button("Submit", variant="primary")
            clear_btn = gr.Button("Clear", variant="secondary")

        with gr.Column(scale=2):
            answer_output = gr.Textbox(label="🧠 AI 답변", lines=8, interactive=False, elem_id="answer_box")
            source_output = gr.Textbox(label="📄 사용된 문서 일부", lines=10, interactive=False, elem_id="source_box")

    submit_btn.click(fn=answer_query, inputs=query_input, outputs=[answer_output, source_output])
    clear_btn.click(lambda: ("", ""), inputs=None, outputs=[answer_output, source_output])

demo.launch(share=True)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device set to use cuda:0


📏 FAISS index dimension: 1536
📏 Embedding model output dimension: 1536
* Running on local URL:  http://127.0.0.1:7876
* Running on public URL: https://11e63dbb5baf012bb7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# ✅ 환경변수
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OPENAI_API_KEY"] = ""  # 🔁 여기에 실제 키 입력

# ✅ 모델 로드
model_path = "./qlora_openchat_familylaw/peft_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to(device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

# ✅ FAISS 벡터 DB 로드
embedding = OpenAIEmbeddings()
vectordb = FAISS.load_local("law_case_linked_db", embeddings=embedding, allow_dangerous_deserialization=True)

# ✅ 차원 확인
print(f"📏 FAISS index dimension: {vectordb.index.d}")
test_vec = embedding.embed_documents(["차원 테스트"])
embed_dim = len(test_vec[0])
print(f"📏 Embedding model output dimension: {embed_dim}")
if vectordb.index.d != embed_dim:
    print("⚠️ FAISS 인덱스 차원과 임베딩 출력 차원이 일치하지 않습니다.")

# ✅ 검색기 설정
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
당신은 '가족법 전문 AI 상담사'입니다. 다음 질문에 대해 아래 형식으로 1000자 이내로 응답하세요.

1️⃣ 법적 근거 요약:
- 관련 법률 조항의 간단한 설명

2️⃣ 관련 판례:
- 질문과 관련된 실제 판례 또는 문서 내용을 간결하게 인용 (출처 미표기)

3️⃣ 결론:
- 사용자의 상황에 따라 고려할 수 있는 법적 방향과 조언을 제시

❓질문:
{question}

📄문서:
{context}

💬답변:"""
)

# ✅ 응답 함수
def answer_query(query):
    docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in docs])
    full_prompt = prompt.format(question=query, context=context)
    
    output = pipe(full_prompt, max_new_tokens=1024, do_sample=True, temperature=0.7)[0]["generated_text"]
    answer = output[len(full_prompt):].strip()

    # ✅ "출처: 미상" 제거
    sources = "\n\n".join([
        f"🔹 [출처: {doc.metadata['source']}]\n{doc.page_content[:500]}..."
        if doc.metadata.get("source") and doc.metadata["source"] != "미상"
        else f"🔹 {doc.page_content[:500]}..."
        for doc in docs
    ])

    return answer, sources

# ✅ Gradio UI
with gr.Blocks(theme=gr.themes.Base(), css="""
#title {font-size: 28px; font-weight: bold; text-align: center;}
#desc {text-align: center; margin-bottom: 20px;}
#query_box textarea {font-size: 16px;}
#answer_box, #source_box {font-size: 15px; line-height: 1.6;}
""") as demo:

    gr.Markdown("<div id='title'>📚 가족법 RAG - OpenAI 임베딩 + 파인튜닝 모델</div>")
    gr.Markdown("<div id='desc'>파인튜닝된 모델 + 기존 벡터 DB(OpenAI Embedding) 기반 실시간 응답</div>")

    with gr.Row():
        with gr.Column(scale=1):
            query_input = gr.Textbox(label="💬 질문", lines=3, placeholder="예: 이혼 시 위자료 기준은 어떻게 정하나요?", elem_id="query_box")
            submit_btn = gr.Button("Submit", variant="primary")
            clear_btn = gr.Button("Clear", variant="secondary")

        with gr.Column(scale=2):
            answer_output = gr.Textbox(label="🧠 AI 답변", lines=8, interactive=False, elem_id="answer_box")
            source_output = gr.Textbox(label="📄 사용된 문서 일부", lines=10, interactive=False, elem_id="source_box")

    submit_btn.click(fn=answer_query, inputs=query_input, outputs=[answer_output, source_output])
    clear_btn.click(lambda: ("", ""), inputs=None, outputs=[answer_output, source_output])

demo.launch(share=True)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device set to use cuda:0


📏 FAISS index dimension: 1536
📏 Embedding model output dimension: 1536
* Running on local URL:  http://127.0.0.1:7877
* Running on public URL: https://ede5f632f893101989.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# ✅ 환경변수 설정
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OPENAI_API_KEY"] = ""  # 🔁 여기에 실제 키 입력

# ✅ 모델 로드
model_path = "./qlora_openchat_familylaw/peft_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to(device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

# ✅ FAISS 벡터 DB 로드
embedding = OpenAIEmbeddings()
vectordb = FAISS.load_local("law_case_linked_db", embeddings=embedding, allow_dangerous_deserialization=True)

# ✅ 차원 확인
print(f"\n📏 FAISS index dimension: {vectordb.index.d}")
test_vec = embedding.embed_documents(["차원 테스트"])
embed_dim = len(test_vec[0])
print(f"📏 Embedding model output dimension: {embed_dim}")
if vectordb.index.d != embed_dim:
    print("⚠️ FAISS 인덱스 차원과 임베딩 출력 차원이 일치하지 않습니다.")

# ✅ 검색기 설정
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# ✅ 프롬프트 템플릿
prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
당신은 '가족법 전문 AI 상담사'입니다. 사용자의 질문에 대해 다음 형식으로 1000자 이내로 응답하세요:

1️⃣ 법적 근거 요약  
- 관련 조항 또는 법 원칙을 간결히 설명

2️⃣ 관련 판례  
- 실제 판례나 문서에서 중요한 부분 요약

3️⃣ 결론 및 조언  
- 사용자의 상황에 맞춘 실용적인 결론과 조언 제시

---

️❓질문:
{question}

📄참고 문서:
{context}

💬답변:
"""
)

# ✅ 질문 응답 함수

def answer_query(query):
    docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in docs])
    full_prompt = prompt.format(question=query, context=context)
    output = pipe(full_prompt, max_new_tokens=1024, do_sample=True, temperature=0.7)[0]["generated_text"]
    answer = output[len(full_prompt):].strip()

    sources = "\n\n".join([
        f"🔹 {doc.page_content[:500]}..."
        for doc in docs
    ])
    return answer, sources

# ✅ Gradio UI
with gr.Blocks(theme=gr.themes.Base(), css="""
#title {font-size: 28px; font-weight: bold; text-align: center;}
#desc {text-align: center; margin-bottom: 20px;}
#query_box textarea {font-size: 16px;}
#answer_box, #source_box {font-size: 15px; line-height: 1.6;}
""") as demo:

    gr.Markdown("<div id='title'>📚 가족법 RAG - OpenAI 임베딩 + 파인튜닉 모델</div>")
    gr.Markdown("<div id='desc'>파인튜닉된 모델 + 기존 벡터 DB(OpenAI 임베딩) 기반 실시간 응답</div>")

    with gr.Row():
        with gr.Column(scale=1):
            query_input = gr.Textbox(label="💬 질문", lines=3, placeholder="예: 이혼 시 위자료 기준은 어떻게 되나요?", elem_id="query_box")
            submit_btn = gr.Button("Submit", variant="primary")
            clear_btn = gr.Button("Clear", variant="secondary")

        with gr.Column(scale=2):
            answer_output = gr.Textbox(label="🤠 AI 답변", lines=10, interactive=False, elem_id="answer_box")
            source_output = gr.Textbox(label="📄 사용된 문서 일반", lines=10, interactive=False, elem_id="source_box")

    submit_btn.click(fn=answer_query, inputs=query_input, outputs=[answer_output, source_output])
    clear_btn.click(lambda: ("", ""), inputs=None, outputs=[answer_output, source_output])

# ✅ 실행
if __name__ == "__main__":
    demo.launch(share=True)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Device set to use cuda:0



📏 FAISS index dimension: 1536
📏 Embedding model output dimension: 1536
* Running on local URL:  http://127.0.0.1:7881
* Running on public URL: https://cccde0f282eeb656b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
